In [1]:
from scipy import io, signal
import matplotlib.pyplot as plt
import dtcwt
import numpy as np
import itertools
import pywt

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
from numpy.linalg import LinAlgError
from statsmodels.tsa.stattools import adfuller
#1
def AE(x): # Absolute Energy
    x = np.asarray(x)
    return sum(x * x)

#2
def SM2(y):
    #t1 = time.time()
    f, Pxx_den = signal.welch(y)
    sm2 = 0
    n = len(f)
    for i in range(0,n):
        sm2 += Pxx_den[i]*(f[i]**2)
    
    #t2 = time.time()
    #print('time: ', t2-t2)
    return sm2


#3
def LOG(y):
    n = len(y)
    return np.exp(np.sum(np.log(np.abs(y)))/n)

#4
def WL(x): # WL in primary manuscript
    return np.sum(abs(np.diff(x)))

#5
def ADF(x): # teststat, pvalue, usedlag
    # augmented dickey fuller
    # returns a tuple
    res = None
    try:
        res = adfuller(x)
    except LinAlgError:
        res = np.NaN, np.NaN, np.NaN
    except ValueError:  # occurs if sample size is too small
        #print('Length Error')
        res = np.NaN, np.NaN, np.NaN

    return res[0], res[1], res[2]


#6
def AC(x, lag=5): # autocorrelation

    """
     [1] https://en.wikipedia.org/wiki/Autocorrelation#Estimation

    """
    # This is important: If a series is passed, the product below is calculated
    # based on the index, which corresponds to squaring the series.
    if type(x) is pd.Series:
        x = x.values
    if len(x) < lag:
        return np.nan
    # Slice the relevant subseries based on the lag
    y1 = x[:(len(x)-lag)]
    y2 = x[lag:]
    # Subtract the mean of the whole series x
    x_mean = np.mean(x)
    # The result is sometimes referred to as "covariation"
    sum_product = np.sum((y1-x_mean)*(y2-x_mean))
    # Return the normalized unbiased covariance
    return sum_product / ((len(x) - lag) * np.var(x))

#7
def BE(x, max_bins=30): # binned entropy
    hist, bin_edges = np.histogram(x, bins=max_bins)
    probs = hist / len(x)
    return - np.sum(p * np.math.log(p) for p in probs if p != 0)

#8
def C3(x, lag = 5): # c3 feature
    n = len(x)
    x = np.asarray(x)
    if 2 * lag >= n:
        return 0
    else:
        return np.mean((np.roll(x, 2 * -lag) * np.roll(x, -lag) * x)[0:(n - 2 * lag)])
    
    
#9
def CC(x, normalize=True): # cid ce
    x = np.asarray(x)
    if normalize:
        s = np.std(x)
        if s!=0:
            x = (x - np.mean(x))/s
        else:
            return 0.0

    x = np.diff(x)
    return np.sqrt(np.sum((x * x)))


#10
def CAM(x): # count above mean
    x = np.asarray(x)
    m = np.mean(x)
    return np.where(x > m)[0].shape[0]

#11
def CBM(x): # count below mean
    x = np.asarray(x)
    m = np.mean(x)
    return np.where(x < m)[0].shape[0]


#12
def AAC(x): #AAC in primary manuscript
    return np.mean(abs(np.diff(x)))

#13
def MSDC(x): # mean second derivative central
    diff = (np.roll(x, 1) - 2 * np.array(x) + np.roll(x, -1)) / 2.0
    return np.mean(diff[1:-1])

#14
def ZC(x, m = 0): # zero/mean crossing
    # m = np.mean(x)
    x = np.asarray(x)
    x = x[x != m]
    return sum(np.abs(np.diff(np.sign(x - m))))/2


#15
def SE(x): # sample entropy
    """
    [1] http://en.wikipedia.org/wiki/Sample_Entropy
    [2] https://www.ncbi.nlm.nih.gov/pubmed/10843903?dopt=Abstract
    """
    x = np.array(x)

    sample_length = 1 # number of sequential points of the time series
    tolerance = 0.2 * np.std(x) # 0.2 is a common value for r - why?

    n = len(x)
    prev = np.zeros(n)
    curr = np.zeros(n)
    A = np.zeros((1, 1))  # number of matches for m = [1,...,template_length - 1]
    B = np.zeros((1, 1))  # number of matches for m = [1,...,template_length]

    for i in range(n - 1):
        nj = n - i - 1
        ts1 = x[i]
        for jj in range(nj):
            j = jj + i + 1
            if abs(x[j] - ts1) < tolerance:  # distance between two vectors
                curr[jj] = prev[jj] + 1
                temp_ts_length = min(sample_length, curr[jj])
                for m in range(int(temp_ts_length)):
                    A[m] += 1
                    if j < n - 1:
                        B[m] += 1
            else:
                curr[jj] = 0
        for j in range(nj):
            prev[j] = curr[j]

    N = n * (n - 1) / 2
    B = np.vstack(([N], B[0]))

    # sample entropy = -1 * (log (A/B))
    similarity_ratio = A / B
    se = -1 * np.log(similarity_ratio)
    se = np.reshape(se, -1)
    return se[0]

#16
def TRAS(x, lag=5):
    # time reversal asymmetry statistic
    """
    |  [1] Fulcher, B.D., Jones, N.S. (2014).
    |  Highly comparative feature-based time-series classification.
    |  Knowledge and Data Engineering, IEEE Transactions on 26, 3026–3037.
    """
    n = len(x)
    x = np.asarray(x)
    if 2 * lag >= n:
        return 0
    else:
        return np.mean((np.roll(x, 2 * -lag) * np.roll(x, 2 * -lag) * np.roll(x, -lag) -
                        np.roll(x, -lag) * x * x)[0:(n - 2 * lag)])
    
    
#17    
def VAR(x): # variance 
    return np.var(x)

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import time
from sklearn.preprocessing import StandardScaler

def load_features(sub):
    # Load DWT-denoised time-series data
    # SubA
    SubA = io.loadmat(str('DWT_' + sub + '.mat'))
    A_sig = SubA[str('DWT_' + sub)]
    A_lab = SubA[str(sub + '_LAB')]

    print(A_sig.shape)
    print(A_lab.shape)

    n_ch, tp, sp = A_sig.shape
    A_sig_dtcwt = np.zeros((sp,(int)(tp/8),n_ch), dtype=np.float32)
    transform = dtcwt.Transform1d()
    for cs in range(sp):
        for ch in range(n_ch):
            vecs_t = transform.forward(A_sig[ch,:,cs], nlevels=3)
            A_sig_dtcwt[cs,:,ch] = np.abs(vecs_t.highpasses[2]).reshape(int(tp/8))
            # Took absoulute value
            # May change

    print(A_sig_dtcwt.shape)

    print(len(vecs_t.highpasses[0]))
    print(len(vecs_t.highpasses[1]))
    print(len(vecs_t.highpasses[2]))

    n_features = 17
    n_features += 2 # ADF
    featuresAll = np.zeros((sp,n_ch*n_features), dtype = np.float32)

    t1 = time.time()
    # To-do: Optimize {PyPy didn't help}
    for cs in range(sp):
        c_f = []
        for ch in range(n_ch):
            y = A_sig_dtcwt[cs,:,ch]
            c_f.append(AE(y))
            c_f.append(SM2(y))
            c_f.append(LOG(y))
            c_f.append(WL(y))
            f1, f2, f3 = ADF(y)
            c_f.append(f1)
            c_f.append(f2)
            c_f.append(f3)
            c_f.append(AC(y))
            c_f.append(BE(y))
            c_f.append(C3(y))
            c_f.append(CC(y))
            c_f.append(CAM(y))
            c_f.append(CBM(y))
            c_f.append(AAC(y))
            c_f.append(MSDC(y))
            c_f.append(ZC(y))
            c_f.append(SE(y))
            c_f.append(TRAS(y))
            c_f.append(VAR(y))
        #c_f = np.array(c_f)
        #c_f = c_f.reshape(n_ch*n_features)
        featuresAll[cs,:] = c_f  

    t2 = time.time()
    print('\ntime: ',t2-t1,'s')


    X_all = featuresAll
    y = A_lab.reshape(len(A_lab),)
    print(np.mean(X_all))
    print(np.var(X_all))

    

    scaler = StandardScaler()
    scaler.fit(X_all)
    #print(scaler.mean_)
    X_all = scaler.transform(X_all)

    print(np.mean(X_all))
    print(np.std(X_all))

    print(X_all.shape)
    X_fs_o = SelectKBest(mutual_info_classif, k=30)
    X_fs = X_fs_o.fit_transform(X_all, y)
    feature_ids = X_fs_o.get_support(indices=True)

    mi = mutual_info_classif(X_fs,y)

    print(X_fs.shape)
    print(y.shape)

    np.save(str('features_' + sub + '.npy'),X_fs)
    np.save(str('y_' + sub + '.npy'),y)

In [4]:
load_features('A')

(5, 1024, 270)
(270, 1)
(270, 128, 5)
512
256
128


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))



time:  81.09680819511414 s
7.4900603
397.74893
2.082096e-09
0.9733286
(270, 95)
(270, 30)
(270,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [5]:
load_features('B')

(5, 1000, 174)
(174, 1)
(174, 125, 5)
500
250
125


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 125, using nperseg = 125
  .format(nperseg, input_length))



time:  41.90011191368103 s
127.06293
263355.3
-3.7039257e-08
0.9733286
(174, 95)
(174, 30)
(174,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [6]:
load_features('C')

(5, 1024, 180)
(180, 1)
(180, 128, 5)
512
256
128


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))



time:  42.46362495422363 s
7.514137
398.84607
-7.027074e-08
0.97332853
(180, 95)
(180, 30)
(180,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [7]:
load_features('D')

(5, 1024, 80)
(80, 1)
(80, 128, 5)
512
256
128


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))



time:  20.486637592315674 s
7.399624
401.22025
4.015471e-08
0.97332853
(80, 95)
(80, 30)
(80,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [8]:
load_features('E')

(5, 1024, 48)
(48, 1)
(48, 128, 5)
512
256
128


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))



time:  11.718890905380249 s
7.4992857
416.83585
-6.6924515e-09
0.97332853
(48, 95)
(48, 30)
(48,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [9]:
load_features('F')

(6, 1024, 80)
(80, 1)
(80, 128, 6)
512
256
128


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))



time:  26.196662425994873 s
7.461227
397.40985
2.0411977e-07
0.97332853
(80, 114)
(80, 30)
(80,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [10]:
load_features('G')

(6, 1024, 120)
(120, 1)
(120, 128, 6)
512
256
128


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))



time:  34.575576066970825 s
7.4959965
397.2147
6.6924514e-08
0.97332853
(120, 114)
(120, 30)
(120,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [11]:
load_features('H')

(6, 768, 150)
(150, 1)
(150, 96, 6)
384
192
96


C:\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 96, using nperseg = 96
  .format(nperseg, input_length))



time:  27.342111110687256 s
5.6762156
224.77808
8.120174e-08
0.97332853
(150, 114)
(150, 30)
(150,)


C:\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


### Classification with all selected features

* Subject A

In [5]:
X = np.load('features_A.npy')
y = np.load('y_A.npy')
print(X.shape)
print(y.shape)

(270, 30)
(270,)


In [13]:
# 10-fold cross validation

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)                                             

Accuracy:  0.8555555555555555 0.09629629629629631


In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.7962962962962963 0.09514987095307503


In [16]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.725925925925926 0.12035612451312537


In [17]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.7925925925925925 0.15608375930152205


In [7]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.762962962962963 0.17276894503245335


* Subject B

In [19]:
X = np.load('features_B.npy')
y = np.load('y_B.npy')
print(X.shape)
print(y.shape)

(174, 30)
(174,)


In [20]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.7666666666666667 0.22244433344430575


In [21]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.6788888888888889 0.24271102565666158


In [22]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.6633333333333333 0.23286725938574354


In [23]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.6777777777777778 0.22355157937705405


In [24]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.6588888888888889 0.25820845249191904


* Subject C

In [25]:
X = np.load('features_C.npy')
y = np.load('y_C.npy')
print(X.shape)
print(y.shape)

(180, 30)
(180,)


In [26]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.8944444444444446 0.15275252316519464


In [27]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.861111111111111 0.2063797291222967


In [28]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.7833333333333333 0.20757268546966


In [29]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.861111111111111 0.15112745009706047


In [30]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.7777777777777777 0.2484519974999766


* Subject D

In [31]:
X = np.load('features_D.npy')
y = np.load('y_D.npy')
print(X.shape)
print(y.shape)

(80, 30)
(80,)


In [32]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.5125 0.3051638903933426


In [33]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.5625 0.20155644370746376


In [34]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.55 0.27838821814150105


In [36]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.625 0.27386127875258304


In [37]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.675 0.406201920231798


* Subject E

In [38]:
X = np.load('features_E.npy')
y = np.load('y_E.npy')
print(X.shape)
print(y.shape)

(48, 30)
(48,)


In [39]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.8916666666666668 0.23629078131263043


In [40]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.8 0.30912061651652345


In [41]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.875 0.3435921354681384


In [42]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.875 0.30956959368344517


In [43]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.85 0.33993463423951903


* Subject F

In [45]:
X = np.load('features_F.npy')
y = np.load('y_F.npy')
print(X.shape)
print(y.shape)

(80, 30)
(80,)


In [46]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.775 0.29154759474226505


In [47]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.625 0.31622776601683794


In [48]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.675 0.2549509756796392


In [49]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.6375 0.361420807370024


In [50]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.575 0.4213074886588179


* Subject G

In [8]:
X = np.load('features_G.npy')
y = np.load('y_G.npy')
print(X.shape)
print(y.shape)

(120, 30)
(120,)


In [52]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.6916666666666667 0.23629078131263043


In [53]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.675 0.21666666666666665


In [54]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.6583333333333333 0.2629955639676584


In [55]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.6416666666666667 0.2891558595482912


In [9]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.625 0.3270236145058097


* Subject H

In [60]:
X = np.load('features_H.npy')
y = np.load('y_H.npy')
print(X.shape)
print(y.shape)

(150, 30)
(150,)


In [61]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2) 

Accuracy:  0.5767857142857143 0.2242049445102787


In [62]:
knn = KNeighborsClassifier(n_neighbors=10)

scores = cross_val_score(knn, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.5375000000000001 0.23915241489690298


In [63]:
clf = tree.DecisionTreeClassifier()

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.5035714285714287 0.23134478201002354


In [64]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.5928571428571427 0.2847304489713536


In [65]:
clf = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: ', scores.mean(), scores.std() * 2)

Accuracy:  0.5821428571428571 0.21455340422679417


### References:
* https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html
* B. C. Ross “Mutual Information between Discrete and Continuous Data Sets”. PLoS ONE 9(2), 2014.
* https://en.wikipedia.org/wiki/Autocorrelation#Estimation
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html
